# Controversial Game Updates vs. Player Engagement and Sentiment

### How do controversial updates or announcements affect player engagement and sentiment in popular online games?

I think this question has some good direction going for it. It's no surprise that players respond to game updates according to whether they like them or not, and this is no different. What I want to explore deeper is how drastically are playercount and reviews affected, and if there are any long term affects on a game's community.

A lot of data may end up being sourced from scraping instead of official APIs, but I'm willing to work with it for the sake of discovery!


In [1]:
# import some important libraries
import requests
import os
import pandas as pd
import time

### Exhibit A: Overwatch 2: Cancellation of PvE mode

This one is going to be a bit tricky, since it wasn't added to Steam until after this controversy. There are still other sources, which I will attempt to use

In [ ]:
#This is the API key for tracker.gg, which I'm using to pull data from
api_key = os.getenv("TRACKER_API_KEY")
headers = {"TRN-Api-Key": api_key}

trackergg_url = "https://api.tracker.gg/api/"

### Exhibit B: Various Runescape updates over the years.

MMOs are a doozy. There's always some update/proposal that gets players *LIVID*. However, I think this will help with my question, since a game like this is subscription based, the impact of these updates might be more dramatic!

In [ ]:
pass